In [1]:
from drowsiness_detection.data import load_raw_60_sec_data
import numpy as np
from drowsiness_detection import config
from joblib import dump
import time
from drowsiness_detection.models import build_lstm_model

In [2]:
data = load_raw_60_sec_data()

data.X_train = np.concatenate([data.X_train, data.X_val])
data.y_train = np.concatenate([data.y_train, data.y_val])


In [3]:
model_name = "lstm"

best_params = {
    "input_shape": (20, 1800, 7),
    "dropout_rate": 0.8953608423789001,
    "learning_rate": 0.0018597842328183852,
    "lstm_units": 82,
    "num_lstm_layers": 1
  }
model = build_lstm_model(**best_params)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1800, 7)]         0         
                                                                 
 lstm (LSTM)                 (None, 82)                29520     
                                                                 
 flatten (Flatten)           (None, 82)                0         
                                                                 
 dropout (Dropout)           (None, 82)                0         
                                                                 
 dense (Dense)               (None, 1)                 83        
                                                                 
Total params: 29,603
Trainable params: 29,603
Non-trainable params: 0
_________________________________________________________________
None


In [4]:
epochs = 25
batch_size = 30
class_weight = {0: 0.84, 1: 1.14}
verbose = 1

history = model.fit(x=data.X_train, y=data.y_train, validation_data=(data.X_test, data.y_test) , verbose=verbose, class_weight=class_weight,
                    epochs=epochs, batch_size=batch_size)


Epoch 1/25
855/855 [==============================] - 635s 740ms/step - loss: 0.6688 - accuracy: 0.6076 - val_loss: 0.6400 - val_accuracy: 0.6941
Epoch 2/25
855/855 [==============================] - 633s 740ms/step - loss: 0.6301 - accuracy: 0.6502 - val_loss: 0.6095 - val_accuracy: 0.7068
Epoch 3/25
855/855 [==============================] - 636s 744ms/step - loss: 0.6314 - accuracy: 0.6420 - val_loss: 0.6275 - val_accuracy: 0.6723
Epoch 4/25
855/855 [==============================] - 651s 762ms/step - loss: 0.6167 - accuracy: 0.6518 - val_loss: 0.6345 - val_accuracy: 0.6658
Epoch 5/25
855/855 [==============================] - 664s 777ms/step - loss: 0.6421 - accuracy: 0.6235 - val_loss: 0.6438 - val_accuracy: 0.6875
Epoch 6/25
855/855 [==============================] - 688s 805ms/step - loss: 0.6463 - accuracy: 0.6278 - val_loss: 0.6422 - val_accuracy: 0.6837
Epoch 7/25
855/855 [==============================] - 693s 810ms/step - loss: 0.6328 - accuracy: 0.6568 - val_loss: 0.6143 -

In [5]:
time_string = time.strftime('%Y%m%d-%H%M%S')
model.save(str(config.MODEL_DIR_PATH.joinpath(f"{model_name}_{time_string}.h5")))


In [6]:
dump(history, str(config.MODEL_DIR_PATH.joinpath(f"histories/{model_name}_{time_string}.joblib")))

INFO:tensorflow:Assets written to: ram://4a57808f-b467-4665-a573-c6aca5f7c70f/assets


INFO:tensorflow:Assets written to: ram://4a57808f-b467-4665-a573-c6aca5f7c70f/assets


['/home/tim/IM/data/models/60sec/histories/lstm_20220729-022018.joblib']

In [7]:
y_pred_test = model.predict(data.X_test)
y_pred_train = model.predict(data.X_train)

In [8]:
print(np.mean((y_pred_test > .5) == data.y_test))


0.4959856660903333


In [9]:
np.save(str(config.PREDICTION_DIR_PATH.joinpath(f"{model_name}_y_pred_test_{time.strftime('%Y%m%d-%H%M%S')}.npy")), y_pred_test)
np.save(str(config.PREDICTION_DIR_PATH.joinpath(f"{model_name}_y_pred_train_{time.strftime('%Y%m%d-%H%M%S')}.npy")), y_pred_train)
